# 核对日月数据一致

在下面输入核对月开始时间和结束时间,并点击单元格——运行所有

In [ ]:
startd = input('请输入月份开始日期（年四位月日两位）')
endd  = input('请输入月份结束日期（年四位月日两位）')

开始核对数据，并将结果文件输出到“输出/水厂日月数据差异”文件夹下。

In [ ]:
import sys
sys.path.append('./StatLedger/module')
import os
os.environ['NLS_LANG'] = 'SIMPLIFIED CHINESE_CHINA.UTF8'
import LeiJiJiSuan as lj
import tjfxdata as tjfx
import pandas as pd
import numpy as np

shuju_leiji = lj.leijijisuan(startd,endd)
shuju_df_m = tjfx.TjfxData().getdata(startd,endd).query("RECORD_TYPE=='m'")
shuju_hedui = pd.merge(shuju_df_m,shuju_leiji,how='left',on=['QUOTA_DEPT_CODE','QUOTA_CODE'],suffixes=('_m', '_d'))
shuju_hedui['QUOTA_VALUE_m'] = pd.to_numeric(shuju_hedui['QUOTA_VALUE_m'],errors='coercs').fillna(0)
shuju_hedui.eval('diff=QUOTA_VALUE_m-QUOTA_VALUE_d',inplace = True)
shuju_hedui['diff'] = pd.to_numeric(shuju_hedui['diff'],errors='coercs').fillna(0)

shuju_hedui.info()
#result = shuju_hedui.query("QUOTA_DEPT_CODE == '1016'")

shuju_hedui_filter = shuju_hedui.query('abs(diff)>0.01')\
[['QUOTA_DATE','QUOTA_DEPT_CODE','QUOTA_CODE','GROUP_NAME_m','QUOTA_NAME_m','QUOTA_VALUE_m','QUOTA_VALUE_d','diff']]
#过滤不显示的指标
shuju_hedui_filter = shuju_hedui_filter[~ shuju_hedui_filter['QUOTA_NAME_m'].str.contains(r'\w单价$|送水泵组平均压力')]


#shuju_hedui.to_excel(r'C:\Users\XieJie\Desktop\水厂日月数据核对'+startd+'.xls')
shuju_hedui_filter.to_excel('./输出/水厂日月数据差异/水厂日月数据差异'+startd+'.xls')
